In [2]:
import geopandas as gpd
import pandas as pd
import csv

In [3]:
#load the tomnod geojson file
tomnod = gpd.read_file('digitalglobe_crowdsourcing_hurricane_harvey_20170915.geojson')

# Explore the tomnod geojson dataframe

In [4]:
tomnod.columns

Index(['id', 'tag_id', 'label', 'tagger_id', 'type_id', 'score', 'agreement',
       'chip_size', 'chip_url', 'timestamp', 'overlay_id', 'acquisition_date',
       'sensor', 'catalog_id', 'map_id', 'geometry'],
      dtype='object')

In [5]:
tomnod.head(30)
tomnod.label.unique()

array(['Flooded / Damaged Building', 'Flooded / Blocked Road',
       'Blocked Bridge', 'Trash Heap'], dtype=object)

In [5]:
tomnod['chip_url'][10]

'https://s3.amazonaws.com/explorationlab/chips/3f97b4225a5e809b9a38b184d19735b3.jpg'

In [6]:
tomnod['agreement'].head()

0    32
1    24
2    20
3    28
4    25
Name: agreement, dtype: int64

In [7]:
tomnod.describe()

,tag_id,tagger_id,type_id,agreement,overlay_id,map_id
count,21603.000000,2.160300e+04,21603.000000,21603.000000,21603.000000,21603.000000
mean,293041.581308,8.685588e+06,234.561218,11.384391,60072.794103,114232.344998
std,170747.173820,8.992336e+06,118.689696,10.091329,57.557200,158989.955620
min,59.000000,2.540000e+02,99.000000,2.000000,60029.000000,3.000000
25%,139180.000000,2.641444e+06,226.000000,6.000000,60043.000000,24188.000000
50%,296018.000000,5.366321e+06,226.000000,9.000000,60043.000000,31082.000000
75%,470733.500000,8.005183e+06,226.000000,13.000000,60045.000000,60962.000000
max,538400.000000,2.893644e+07,1870.000000,166.000000,60179.000000,432999.000000


In [8]:
tomnod.head(5)

,id,tag_id,label,tagger_id,type_id,score,agreement,chip_size,chip_url,timestamp,overlay_id,acquisition_date,sensor,catalog_id,map_id,geometry
0,214149-59,59,Flooded / Damaged Building,30425,226,1.000000,32,,https://s3.amazonaws.com/explorationlab/chips/...,2017-09-15T22:01:52,60044,2017-08-31T23:39:39,GE01,105001000B95E200,9029,POINT (-95.096374 29.84318)
1,214149-72,72,Flooded / Damaged Building,6294177,226,1.000000,24,,https://s3.amazonaws.com/explorationlab/chips/...,2017-09-15T22:01:52,60043,2017-08-31T23:39:05,GE01,105001000B95E100,33640,POINT (-95.50154999999999 29.760742)
2,214149-76,76,Flooded / Damaged Building,6294177,226,1.000000,20,,https://s3.amazonaws.com/explorationlab/chips/...,2017-09-15T22:01:52,60043,2017-08-31T23:39:05,GE01,105001000B95E100,28027,POINT (-95.50540700000001 29.75222)
3,214149-108,108,Flooded / Damaged Building,6294177,226,1.000000,28,,https://s3.amazonaws.com/explorationlab/chips/...,2017-09-15T22:01:52,60043,2017-08-31T23:39:05,GE01,105001000B95E100,32835,POINT (-95.521941 29.748386)
4,214149-111,111,Flooded / Damaged Building,6294177,226,1.000000,25,,https://s3.amazonaws.com/explorationlab/chips/...,2017-09-15T22:01:52,60043,2017-08-31T23:39:05,GE01,105001000B95E100,32835,POINT (-95.52256800000001 29.748596)


In [ ]:
tomnod.label.unique()

In [ ]:
tomnod.label.value_counts()

In [ ]:
type(tomnod['geometry'])

In [ ]:
tomnod_x = tomnod['geometry'].x

In [ ]:
type(tomnod['geometry'].x)

In [ ]:
tomnod_y = tomnod['geometry'].y

In [ ]:
30.090656 in tomnod_x

In [ ]:
tomnod['tomnod_x'] = tomnod_x

In [ ]:
tomnod['tomnod_y'] = tomnod_y

In [ ]:
def processTup(tup):
    return [float(ele) for ele in (tup.strip('()').split(','))]

# Load the tifRange of catalog_id

In [ ]:
tifRange = pd.read_csv('tifRange-post.csv')

In [ ]:
tifRange.head(5)

In [ ]:
def getTifFromCoor(x, y, catalog):
    for index, row in tifRange.iterrows():
        minxy = processTup(row['minxy'])
        maxxy = processTup(row['maxxy'])
        if x >= minxy[0] and x <= maxxy[0] \
                and y >= minxy[1] and y <= maxxy[1] \
                and catalog == row['catalog_id']:
            return row['tif']

In [ ]:
print(getTifFromCoor(-95.096444,29.845092,'10300100512FDE00'))

In [ ]:
#show the rows with missing catalg_id
tomnod.loc[tomnod['catalog_id'] == '']

In [ ]:
#count the number of row missing catalog_id
tomnod.loc[tomnod['catalog_id'] == ''].count()

In [32]:
#get range of catalog_id
cat_id_range = dict()
for catalog_id in tifRange['catalog_id'].unique():
    for index,row in tifRange.iterrows():
        if row['catalog_id'] == catalog_id:
            minx = processTup(row['minxy'])[0]
            miny = processTup(row['minxy'])[1]
            maxx = processTup(row['maxxy'])[0]
            maxy = processTup(row['maxxy'])[1]
            if catalog_id not in cat_id_range:
                cat_id_range[catalog_id] = ((minx,miny),(maxx,maxy))
            else:
                cat_id_range[catalog_id] = ((min(minx,cat_id_range[catalog_id][0][0]),
                                            min(miny,cat_id_range[catalog_id][0][1])),
                                            (max(maxx,cat_id_range[catalog_id][1][0]),
                                             max(maxy,cat_id_range[catalog_id][1][1])))
                                            
    

In [33]:
for k,v in cat_id_range.items():
    print (k,v)

103001006FCDF300 ((-96.15234375, 28.388671875), (-95.888671875, 30.05859375))
105001000B95E100 ((-95.712890625, 29.267578125), (-95.2734375, 30.234375))
1020010067290D00 ((-95.09765625, 29.00390625), (-94.658203125, 29.794921875))
105001000B920A00 ((-95.537109375, 29.53125), (-95.2734375, 30.234375))
103001006D44B500 ((-95.625, 28.30078125), (-95.009765625, 29.267578125))
1040010030BE1200 ((-96.50390625, 28.212890625), (-96.15234375, 29.267578125))
105001000B9D8000 ((-95.2734375, 29.53125), (-95.009765625, 30.234375))
10400100319BA200 ((-96.767578125, 27.94921875), (-96.50390625, 28.388671875))
10200100682E8400 ((-95.185546875, 29.091796875), (-94.833984375, 29.794921875))
103001006F018C00 ((-95.9765625, 28.388671875), (-95.625, 30.05859375))
1020010067E38000 ((-95.44921875, 29.00390625), (-95.009765625, 29.794921875))
1.04E+12 ((-97.03125, 28.30078125), (-96.85546875, 29.1796875))
1030010071D49400 ((-95.361328125, 29.8828125), (-95.009765625, 31.025390625))
105001000B920900 ((-95.4492

In [34]:
#do not run this cell
for k,v in cat_id_range.items():
    print (k,v)

103001006FCDF300 ((-96.15234375, 28.388671875), (-95.888671875, 30.05859375))
105001000B95E100 ((-95.712890625, 29.267578125), (-95.2734375, 30.234375))
1020010067290D00 ((-95.09765625, 29.00390625), (-94.658203125, 29.794921875))
105001000B920A00 ((-95.537109375, 29.53125), (-95.2734375, 30.234375))
103001006D44B500 ((-95.625, 28.30078125), (-95.009765625, 29.267578125))
1040010030BE1200 ((-96.50390625, 28.212890625), (-96.15234375, 29.267578125))
105001000B9D8000 ((-95.2734375, 29.53125), (-95.009765625, 30.234375))
10400100319BA200 ((-96.767578125, 27.94921875), (-96.50390625, 28.388671875))
10200100682E8400 ((-95.185546875, 29.091796875), (-94.833984375, 29.794921875))
103001006F018C00 ((-95.9765625, 28.388671875), (-95.625, 30.05859375))
1020010067E38000 ((-95.44921875, 29.00390625), (-95.009765625, 29.794921875))
1.04E+12 ((-97.03125, 28.30078125), (-96.85546875, 29.1796875))
1030010071D49400 ((-95.361328125, 29.8828125), (-95.009765625, 31.025390625))
105001000B920900 ((-95.4492

In [35]:
#process tomnod further to recover the missing catalog id
post_event_catalog = ['105001000B95E200', '105001000B95E100', '1040010032211E00']
for index, row in tomnod.iterrows():
    if row['catalog_id'] in post_event_catalog:
        s = row['catalog_id']
        #row['complete_catalog_id'] = s
        tomnod.set_value(index,'complete_catalog_id',s)
        #print("found existing post catalog", s)
    elif row['catalog_id'] == '':
        for k,v in cat_id_range.items():
            if row['tomnod_x'] >= v[0][0] and row['tomnod_x'] <= v[1][0] and \
                     row['tomnod_y'] >= v[0][1] and row['tomnod_y'] <= v[1][1]:
                #row['complete_catalog_id'] = k
                tomnod.set_value(index,'complete_catalog_id',k)
                print(index)
            


16032
16032
16032
16033
16033
16033
16034
16038
16039
16040
16040
16040
16041
16042
16043
16044
16045
16046
16048
16048
16048
16052
16052
16053
16053
16055
16055
16056
16056
16057
16058
16058
16059
16060
16060
16067
16068
16069
16070
16070
16070
16071
16071
16071
16072
16072
16072
16075
16075
16075
16110
16110
16110
16113
16113
16113
16115
16115
16115
16119
16119
16119
16122
16122
16122
16123
16123
16123
16127
16132
16132
16132
16133
16134
16134
16134
16140
16140
16140
16144
16146
16147
16147
16147
16154
16154
16154
16155
16155
16155
16156
16156
16158
16158
16163
16163
16164
16164
16164
16166
16166
16169
16171
16171
16171
16172
16172
16173
16174
16174
16175
16176
16176
16176
16177
16178
16178
16178
16179
16179
16186
16187
16187
16187
16188
16188
16188
16189
16189
16189
16190
16190
16190
16191
16191
16191
16192
16193
16194
16195
16196
16197
16198
16199
16200
16201
16202
16203
16204
16205
16206
16207
16207
16207
16208
16209
16210
16210
16210
16211
16212
16213
16214
16214
16214
16215
1621

16659
16659
16659
16659
16660
16660
16660
16660
16660
16660
16660
16660
16660
16660
16661
16661
16661
16661
16661
16661
16662
16662
16662
16662
16662
16662
16663
16664
16665
16665
16665
16665
16665
16665
16666
16666
16666
16666
16666
16666
16667
16667
16667
16667
16667
16667
16668
16668
16668
16668
16668
16668
16669
16669
16669
16669
16669
16669
16670
16670
16670
16670
16670
16670
16671
16671
16671
16671
16671
16671
16672
16672
16672
16672
16672
16672
16673
16673
16673
16673
16673
16673
16674
16674
16674
16674
16674
16674
16675
16675
16675
16675
16675
16675
16677
16677
16677
16677
16677
16677
16678
16678
16678
16678
16678
16678
16679
16679
16679
16679
16679
16679
16680
16680
16680
16680
16680
16680
16681
16681
16681
16681
16681
16681
16682
16682
16682
16682
16682
16682
16683
16683
16683
16683
16683
16683
16684
16685
16685
16685
16685
16685
16685
16686
16686
16686
16686
16686
16686
16688
16689
16689
16689
16689
16689
16689
16690
16691
16691
16691
16691
16691
16691
16692
16692
16692
1669

17373
17374
17374
17375
17375
17376
17376
17377
17377
17378
17378
17379
17379
17380
17380
17381
17381
17382
17382
17383
17383
17384
17384
17385
17385
17386
17386
17387
17387
17388
17388
17389
17389
17390
17390
17391
17391
17391
17392
17392
17392
17393
17393
17393
17394
17394
17394
17395
17395
17395
17396
17396
17396
17397
17397
17397
17398
17398
17398
17399
17399
17399
17400
17400
17401
17401
17402
17402
17402
17403
17403
17404
17404
17404
17405
17405
17405
17406
17406
17406
17407
17407
17407
17408
17408
17408
17409
17409
17409
17410
17410
17410
17411
17411
17412
17412
17412
17413
17413
17413
17414
17414
17415
17415
17416
17417
17418
17418
17419
17419
17420
17420
17420
17421
17421
17421
17422
17422
17422
17423
17423
17423
17424
17424
17424
17425
17425
17426
17426
17427
17427
17427
17428
17428
17429
17429
17429
17430
17430
17430
17431
17431
17431
17432
17432
17432
17433
17433
17433
17434
17434
17435
17435
17435
17436
17436
17436
17437
17437
17437
17438
17438
17438
17439
17439
17439
1744

18859
18860
18861
18862
18863
18864
18865
18866
18867
18868
18869
18870
18871
18872
18873
18874
18875
18876
18877
18878
18879
18880
18881
18882
18883
18884
18885
18886
18887
18888
18889
18890
18891
18892
18893
18894
18895
18896
18897
18898
18899
18900
18901
18902
18903
18904
18905
18906
18907
18908
18909
18910
18911
18912
18913
18914
18915
18916
18917
18918
18919
18920
18921
18922
18923
18924
18925
18926
18927
18928
18929
18930
18931
18932
18933
18934
18935
18936
18937
18938
18939
18940
18941
18942
18943
18944
18945
18946
18947
18948
18949
18950
18951
18952
18953
18954
18955
18956
18957
18958
18959
18960
18961
18962
18963
18964
18965
18966
18967
18968
18969
18970
18971
18972
18973
18974
18975
18976
18977
18978
18979
18980
18981
18982
18983
18984
18985
18986
18987
18988
18989
18990
18991
18992
18993
18994
18995
18996
18997
18998
18999
19000
19001
19002
19003
19004
19005
19006
19007
19008
19009
19010
19011
19012
19013
19014
19015
19016
19017
19018
19019
19020
19021
19022
19023
19024
1902

20563
20564
20565
20566
20567
20568
20569
20570
20571
20572
20573
20574
20575
20576
20577
20578
20579
20580
20581
20582
20583
20584
20585
20586
20587
20588
20589
20590
20591
20592
20593
20594
20595
20596
20597
20598
20599
20600
20601
20602
20603
20604
20605
20606
20607
20608
20609
20610
20611
20612
20613
20614
20615
20616
20617
20618
20619
20620
20621
20622
20623
20624
20625
20626
20627
20628
20629
20630
20631
20632
20633
20634
20635
20636
20637
20638
20639
20640
20641
20642
20643
20644
20645
20646
20647
20648
20649
20650
20651
20652
20653
20654
20655
20656
20657
20658
20659
20660
20661
20662
20663
20664
20665
20666
20667
20668
20669
20670
20671
20672
20673
20674
20675
20676
20677
20678
20679
20680
20681
20682
20683
20684
20685
20686
20687
20688
20689
20690
20691
20692
20693
20694
20695
20696
20697
20698
20699
20700
20701
20702
20703
20704
20705
20706
20707
20708
20709
20710
20711
20712
20713
20714
20715
20716
20717
20718
20719
20720
20721
20722
20723
20724
20725
20726
20727
20728
2072

In [38]:
#show the rows with missing catalog_id
tomnod.loc[tomnod['catalog_id'] == '']
#we see that there is no more missing catalog_id

,id,tag_id,label,tagger_id,type_id,score,agreement,chip_size,chip_url,timestamp,overlay_id,acquisition_date,sensor,catalog_id,map_id,geometry,tomnod_x,tomnod_y,complete_catalog_id


In [ ]:
#show the rows with missing catalg_id
tomnod.loc[tomnod['complete_catalog_id'] == '']

# Now we can save a complete tomnod post-event data

In [41]:
#be careful when uncomment this
#tomnod.to_csv('tomnod_complete-1.csv', encoding='utf-8')

In [9]:
#be careful when uncomment this
#tomnod.to_csv('tomnod.csv', encoding='utf-8')

In [40]:
#see the previously missing catalog_id
tomnod.complete_catalog_id.unique()

array(['105001000B95E200', '105001000B95E100', '1040010032211E00',
       '105001000B9D8100', '1030010070C13600', '10400100324DAE00',
       '105001000B9D7F00', '1020010068D84400', '1020010068D6F400'], dtype=object)

In [43]:
#show the rows with missing catalg_id
tomnod.loc[tomnod['complete_catalog_id'] == '']

,id,tag_id,label,tagger_id,type_id,score,agreement,chip_size,chip_url,timestamp,overlay_id,acquisition_date,sensor,catalog_id,map_id,geometry,tomnod_x,tomnod_y,complete_catalog_id


# Below is to make the complete table of pre and post event
# But we need the tifRange for pre-event also
# This operation is costly 

In [54]:
#be careful with the output file name 
with open('coordinateAndTif1.csv','w') as myFile:
    writer = csv.writer(myFile)
    writer.writerow(['post_catalog_id','post_tif','pre_catalog_id','pre_tif','min_xy','max_xy','x_coord', 'y_coord', 'label'])
    for index, row in tomnod.iterrows():
        for index_t, row_t in tifRange.iterrows():
            minxy = processTup(row_t['minxy'])
            maxxy = processTup(row_t['maxxy'])
            if row['tomnod_x'] >= minxy[0] and row['tomnod_x'] <= maxxy[0] \
                  and row['tomnod_y'] >= minxy[1] and row['tomnod_y'] <= maxxy[1]:
                temp = [row['complete_catalog_id'], getTifFromCoor(row['tomnod_x'],row['tomnod_y'],row['complete_catalog_id']),
                        row_t['catalog_id'],getTifFromCoor(row['tomnod_x'],row['tomnod_y'],row_t['catalog_id']),
                        row_t['minxy'], row_t['maxxy'], 
                        row['tomnod_x'], row['tomnod_y']]
                if row['complete_catalog_id'] == row_t['catalog_id']:
                    temp.append(row['label'])
                else:
                    temp.append('non-damaged '+ row['label'])
                writer.writerow(temp)
        

In [62]:
tifRange['minxy'][0].strip('()').split(',')

['-95.712890625', ' 29.53125']

In [63]:
# def processTup(tup):
#     return [float(ele) for ele in (tup.strip('()').split(','))]

In [64]:
a = processTup(tifRange['minxy'][0])

In [65]:
a

[-95.712890625, 29.53125]

In [70]:
b = [1,a,2,3]
b

[1, [-95.712890625, 29.53125], 2, 3]

# below are just some web API to display geojson


In [8]:
import geojsonio

In [11]:
tomnod_geo = tomnod[['geometry', 'id']]
#tomnod_geo
geo = tomnod_geo.to_json()
print(geo)
#type(geo) #string

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"id": "212904-35"}, "geometry": {"type": "Point", "coordinates": [-95.096444, 29.845092]}}, {"id": "1", "type": "Feature", "properties": {"id": "212904-59"}, "geometry": {"type": "Point", "coordinates": [-95.096374, 29.84318]}}, {"id": "2", "type": "Feature", "properties": {"id": "212904-72"}, "geometry": {"type": "Point", "coordinates": [-95.50155, 29.760742]}}, {"id": "3", "type": "Feature", "properties": {"id": "212904-98"}, "geometry": {"type": "Point", "coordinates": [-95.519972, 29.741735]}}, {"id": "4", "type": "Feature", "properties": {"id": "212904-101"}, "geometry": {"type": "Point", "coordinates": [-95.519494, 29.741475]}}, {"id": "5", "type": "Feature", "properties": {"id": "212904-108"}, "geometry": {"type": "Point", "coordinates": [-95.521941, 29.748386]}}, {"id": "6", "type": "Feature", "properties": {"id": "212904-111"}, "geometry": {"type": "Point", "coordinates": [-95.522568, 29.7

In [10]:
geojsonio.display(geo)

'http://geojson.io/#id=gist:/9bad57816e15e30fc7ef13e620186791'

In [8]:
tomnod_to_json = tomnod.to_json()
#import json
#print(tomnod_to_json)

In [9]:
geojsonio.display(tomnod_to_json)

'http://geojson.io/#id=gist:/c46034f7475818edbb7b66156c9bda9e'

In [31]:
test_raw = open('raw_results.geojson').read()
geojsonio.display(test_raw)

'http://geojson.io/#id=gist:/930da5c3964b7e85377e4e6a97625a4b'

In [12]:
countries = open('countries.geojson').read()
geojsonio.display(countries)

'http://geojson.io/#id=gist:/abba2efb5ce52762ab7ed25b37dbaa62'

In [23]:
countriesdf = gpd.read_file('countries.geojson')
type(countries)

str

In [14]:
countriesdf.columns

Index(['geometry', 'id', 'name'], dtype='object')

In [15]:
countriesdf.head()

,geometry,id,name
0,"POLYGON ((61.210817 35.650072, 62.230651 35.27...",AFG,Afghanistan
1,"(POLYGON ((16.326528 -5.87747, 16.57318 -6.622...",AGO,Angola
2,"POLYGON ((20.590247 41.855404, 20.463175 41.51...",ALB,Albania
3,"POLYGON ((51.579519 24.245497, 51.757441 24.29...",ARE,United Arab Emirates
4,"(POLYGON ((-65.5 -55.2, -66.45 -55.25, -66.959...",ARG,Argentina


In [28]:
geomecountry = countriesdf['geometry']
test = geomecountry.to_json()
geojsonio.display(test)

'http://geojson.io/#id=gist:/42abccb8225cfe8f6d11a3b0016b45bc'